# 基于graph copy的分布式训练demo

本demo用于演示在使用基于graph copy的分布式训练时，如何加载之前保存的checkpoint，并继续训练。

注意在恢复参数的时候，graph结构没有变化。如果需要重新加载graph，请参考`GraphCopyCheckpointDemo.ipynb`。

In [1]:
import os
import pandas as pd
import tensorflow as tf
import sys
sys.path.insert(0, "../../")

from rcalgo.tf.models.classifier.text_classifier import TransformerClassifier
from rcalgo.tf.utils.tokenizers import CharTokenizer

In [2]:
config = {
    "learning_rate": 0.0001,

    "num_classes": 2,
    "max_seq_len": 100,
    "embedding_size": 256,
    "num_layers": 2,
    "conv_num_filters": 256,
    # "keep_prob": 0.5,

    "gpu_id": 0,
    "gpu_num": 2,

    "model_output_dir": "./saved_models",
    "model_name": "transformer",

    "clip_type": "clip_value",
    "clip_gradient": 1.0
}

In [3]:
# 1. load data
data_dir = "/home/web_server/antispam/project/datasets/female_vulgar_small/"
train_data = pd.read_csv(data_dir + 'train.csv')

In [4]:
# count word dict
tokenizer = CharTokenizer.build_from_corpus(train_data['text'], freq_threshold=1)
word_dict = tokenizer.word2idx

In [5]:
# 2. create model
model = TransformerClassifier(config, word_dict, name="default",
                         distribute="tf_distribute")
model.build_model()
print(model.model_summary())

2020-11-04 18:43:24,358  model.py (73) (MainThread) : INFO  config: {'learning_rate': 0.0001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 256, 'num_layers': 2, 'conv_num_filters': 256, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'transformer', 'clip_type': 'clip_value', 'clip_gradient': 1.0}


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
reduction_indices is deprecated, use axis instead

total parameter number: 3303170
                                        variable_name  variable_shape  \
0                   embedding_layer/embedding_table:0     [3398, 256]   
1                mw_conv_block/conv_1d/he_uniform/W:0   [3, 256, 256]   
2                           mw_conv_block/conv_1d/B:0           [256]   
3              mw_conv_block/conv_1d_1/he_uniform/W:0   [4, 256, 256]   
4                         mw_conv_block/conv_1d_1/B:0           [256]   
5              mw_conv_block/conv_1d_2/he_uniform/W:0   [5, 256, 256]   
6                         mw_conv_block/conv_1d_2/B:0           [256]   
7   transform2_block/multi_head_attention/conv_1d/...   [1, 256, 768]   
8   transform2_block/multi_head_attention/conv_1d/B:0           [768]   
9   transform2_block/multi_head_attention/conv_1d_...   [1, 256, 2

In [ ]:
# 3. train
model.train(
    [train_data['text'], train_data['label']],
    batch_size=1024,
    test_size=0.5,
    epochs=1,
    hooks=[])


2020-11-04 18:43:26,455  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-11-04 18:43:26,456  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2


In [ ]:
model_path = "./saved_models/manually_saved_checkpoint.ckpt"
# 保存模型checkpoint的时候会同时保存原图对应的checkpoint: ./saved_models/single_graph/manually_saved_checkpoint.ckpt
# 和graph copy后multi_replica对应的checkpoint: ./saved_models/manually_saved_checkpoint.ckpt
model.dump_checkpoint(model_path)

# 从多图的checkpoint中恢复参数
multi_replica_path = model_path
model.trainer.load_multi_replica_checkpoint(multi_replica_path)

In [ ]:
# 继续训练
print("continue training ...")
model.train(
      [train_data['text'], train_data['label']],
      batch_size=1024,
      test_size=0.5,
      epochs=1,
      hooks=[])

In [ ]:
# 从单图的checkpoint中恢复参数
single_model_path = os.path.join(os.path.dirname(model_path), "single_graph", os.path.basename(model_path))
model.trainer.load_single_graph_checkpoint(single_model_path)

In [ ]:
# 继续训练
print("continue training ...")
model.train(
      [train_data['text'], train_data['label']],
      batch_size=1024,
      test_size=0.5,
      epochs=1,
      hooks=[])